# WINDNODE ABW - Scenario Analysis

<img src="http://reiner-lemoine-institut.de//wp-content/uploads/2015/09/rlilogo.png" width="100" style="float: right">

__copyright__ 	= "© Reiner Lemoine Institut" <br>
__license__ 	= "GNU Affero General Public License Version 3 (AGPL-3.0)" <br>
__url__ 		= "https://www.gnu.org/licenses/agpl-3.0.en.html" <br>
__author__ 		= "Jonathan Amme" <br>

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
####### WINDNODE ########
# define and setup logger
from windnode_abw.tools.logger import setup_logger
logger = setup_logger()

# load configs
from windnode_abw.tools import config
config.load_config('config_data.cfg')
config.load_config('config_misc.cfg')

from windnode_abw.analysis import analysis
from windnode_abw.tools.draw import *

######### DATA ##########

import re
import pandas as pd

####### Plotting ########

# Plotting
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.ticker import ScalarFormatter
import seaborn as sns
# set seaborn style
sns.set()

import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots

In [ ]:
# papermill parameters will be inserted below this tagged cell

TODO: suppress long log output of analysis

In [ ]:
# obtain processed results
regions_scns, results_scns = analysis(run_timestamp=run_timestamp,
                                      scenarios=scenarios,
                                      force_new_results=force_new_results)

In [ ]:
##### Usefull stuff ######

# Names of Municialities
MUN_NAMES = regions_scns['StatusQuo_DSMno'].muns.gen.to_dict()
# extend for total ABW region
MUN_NAMES.update({100:'ABW'})
# Colormap
CMAP = px.colors.sequential.GnBu_r

# 0 Results Overview

In [ ]:
plot_essential_scenario_results(results_scns, scenarios)

# 1 Demand and Generation

## 1.1 Installed Capacities Electricity/Heat

TODO
- [ ] Fix title overlap
- [ ] Scenario order?

In [ ]:
cap_el = pd.DataFrame({scn: results_scns[scn]['parameters']['Installed capacity electricity supply'].sum(axis=0)
                       for scn in scenarios}).T

cap_el = cap_el.assign(sum=cap_el.sum(axis=1)) \
    .sort_values(by='sum', ascending=False) \
    .drop('sum', axis=1) \
    .rename(columns=PRINT_NAMES)

fig = go.Figure()

for tech, data in cap_el.iteritems():
    fig.add_trace(go.Bar(x=cap_el.index,
                         y=data,
                         name=tech,
                         #orientation='h',
                         showlegend=True))

fig.update_layout(
    title='Installed Capacity Electricity Supply',
    barmode='stack',
    hovermode="x unified",
    height=800,
    xaxis_tickfont_size=12,
    yaxis=dict(title='MW',
               titlefont_size=16,
               tickfont_size=14),
    legend=dict(orientation="h",
                yanchor="bottom",
                y=1.02,
                xanchor="right",
                x=1),
    autosize=True)

fig.show()

In [ ]:
cap_th = pd.DataFrame({scn: results_scns[scn]['parameters']['Installed capacity heat supply'].sum(axis=0)
                       for scn in scenarios}).T.rename(columns=PRINT_NAMES)

cap_th = cap_th.assign(sum=cap_th.sum(axis=1)) \
    .sort_values(by='sum', ascending=False) \
    .drop('sum', axis=1) \
    .rename(columns=PRINT_NAMES)

fig = go.Figure()

for tech, data in cap_th.iteritems():
    fig.add_trace(go.Bar(x=cap_th.index,
                         y=data,
                         name=tech,
                         #orientation='h',
                         showlegend=True))

fig.update_layout(
    title='Installed Capacity Heat Supply',
    barmode='stack',
    hovermode="x unified",
    height=800,
    xaxis_tickfont_size=12,
    yaxis=dict(title='MW',
               titlefont_size=16,
               tickfont_size=14),
    legend=dict(orientation="h",
                yanchor="bottom",
                y=1.02,
                xanchor="right",
                x=1),
    autosize=True)
fig.show()

## 1.2 Electricity and Heat Generation

## 1.3 Electricity and Heat Demand

# 2 Area required by RES

## 2.1 Available areas: Scenarios

### 2.1.1 Available areas: Wind Energy

### 2.1.2 Available areas: Ground-Mounted PV

### 2.1.3 Available areas: Roof-Mounted PV

## 2.2 Area usage